In [1]:
import pandas as pd
import os
import json
import requests
import datetime
import itertools

In [2]:
sat_api_url = "https://landsatlook.usgs.gov/sat-api/stac/search"

In [3]:
#checking to see if sat api is there
response = requests.get(sat_api_url)
response

<Response [200]>

In [4]:
#checking for response
response.json()

{'type': 'FeatureCollection',
 'meta': {'page': 1, 'limit': 10, 'found': 16848225, 'returned': 10},
 'features': [{'stac_version': '0.7.0',
   'stac_extensions': ['eo',
    'https://landsat.usgs.gov/stac/landsat-extension/schema.json'],
   'id': 'LC08_L1TP_043023_20201208_20201208_02_RT',
   'type': 'Feature',
   'bbox': [-116.50227094257501,
    52.018564310686244,
    -113.01487319852407,
    54.172509521935666],
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-115.75185001009054, 54.172509521935666],
      [-113.01487319852407, 53.704707426826],
      [-113.87126025518506, 52.018564310686244],
      [-116.50227094257501, 52.47686404038613],
      [-115.75185001009054, 54.172509521935666]]]},
   'description': 'Landsat Collection 2 Level-1 Product',
   'properties': {'datetime': '2020-12-08',
    'eo:cloud_cover': 49.29,
    'eo:sun_azimuth': 166.90056926,
    'eo:sun_elevation': 13.13127626,
    'eo:platform': 'LANDSAT_8',
    'eo:instrument': 'OLI_TIRS',
    'eo:off_nadir'

In [5]:
#this will fetch query/do pangination
def fetch_sat_api(query):
    headers = {
        "Content-Type": "application/json",
        "Accept-Encoding": "gzip",
        "Accept": "application/geo+json",
    }

    url = "https://landsatlook.usgs.gov/sat-api/stac/search/"
    data = requests.post(url, headers=headers, json=query).json()
    error = data.get("message", "")
    if error:
        raise Exception(f"SAT-API failed and returned: {error}")

    meta = data.get("meta", {})
    if not meta.get("found"):
        return []
    print(meta)

    features = data["features"]
    if data["links"]:
        curr_page = int(meta["page"])
        query["page"] = curr_page + 1
        query["limit"] = meta["limit"]

        features = list(itertools.chain(features, fetch_sat_api(query)))

    return features

In [6]:
#query
#getting JSON Decode error? This process worked in ST not sure of the difference here
date_min="2020-01-01"
date_max="2020-10-24"

start = datetime.datetime.strptime(date_min, "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
end = datetime.datetime.strptime(date_max, "%Y-%m-%d").strftime("%Y-%m-%dT23:59:59Z")

query = {
    "time": f"{start}/{end}",
    
    "limit": 500 # We limit to 500 items per Page (requests) to make sure sat-api doesn't fail to return big features collection
}

features = fetch_sat_api(query)

JSONDecodeError: Expecting value: line 1 column 2 (char 1)

This is old stuff from when I had this working. I think the Stac records have changed formats since then so, this should be updated.

In [ ]:
df = pd.DataFrame(features)

ids = []
for item in df['id']:
    ids.append(item)

dates = []
for item in df['properties']:
    dates.append(item['datetime'])

In [ ]:
ids = pd.DataFrame(ids, columns = ['Product ID'])
dates = pd.DataFrame(dates, columns = ['Date'])

In [ ]:
df= pd.concat([ids, dates], axis = 1)

In [ ]:
df

In [ ]:
df.to_csv('IT_List.csv')